### 1.output parser 


In [4]:
# we use to generate detail report on topic to llm 
# then generate summary of detail report through llm

In [5]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct-0905",  # ✅ spelling fixed
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.3,
)

response = llm.invoke("Hi")
print(response.content)


Hi there! How can I help you today?


In [6]:
from langchain_core.prompts import PromptTemplate

In [ ]:
t1=PromptTemplate(
    template="write a detailed report on {topic}",
    input_variables=['topic']
)

In [ ]:
t2=PromptTemplate(
template="write a 5 line summary on the followrin text reoprt on {topic}",
input_variables=['topic']    
)

In [9]:
p1=t1.invoke({'topic':'black hole'})

In [10]:
result=llm.invoke(p1)

In [12]:
result.content

'**Comprehensive Report on Black Holes**  \n*Prepared by: [Your Name]*  \n*Date: [Insert Date]*\n\n---\n\n### **1. Executive Summary**\nBlack holes are regions of spacetime where gravity is so intense that nothing—not even light—can escape. Predicted by Einstein’s general relativity (1915) and confirmed via indirect (1970s) and direct (2019) observations, they represent the ultimate endpoint of massive stellar evolution. This report synthesizes current theoretical frameworks, observational evidence, and unresolved questions.\n\n---\n\n### **2. Historical Development**\n| **Year** | **Milestone** |\n|----------|---------------|\n| 1783     | John Michell proposes “dark stars” (Newtonian gravity). |\n| 1915     | Schwarzschild derives first exact black-hole solution. |\n| 1963     | Kerr solution for rotating black holes. |\n| 1967     | Wheeler coins term “black hole.” |\n| 1974     | Hawking predicts quantum thermal radiation. |\n| 2015     | LIGO detects gravitational waves from black

In [11]:
p2=t2.invoke({"topic":result.content})


In [13]:
re1=llm.invoke(p2)

In [14]:
print(re1.content)

Black holes are ultra-dense spacetime regions where gravity traps everything, predicted by Einstein and now seen from stellar to super-massive scales.  
Key milestones span Michell’s 1783 “dark stars” to the 2019 EHT image of M87*, with LIGO’s 2015 gravitational-wave detection in between.  
Theory fixes their size and spin via Schwarzschild/Kerr metrics, while accretion disks, jets, quasinormal modes and star orbits provide observational proof.  
Quantum gravity confronts Hawking radiation, information loss and possible Planck-scale structure, yet the paradox remains open.  
Next-gen facilities (EHT 2.0, LISA) will map spins, probe horizon micro-structure and test cosmic censorship within the coming decade.


### 2.now using chain

In [15]:
from langchain_core.output_parsers import StrOutputParser

In [16]:

parser=StrOutputParser()

In [21]:
chain=t1 | llm | t2 | llm | parser

# here parser is using for extract string 

In [22]:

chain.invoke({"topic":"black hole"})

'Black holes are extreme-gravity zones where nothing escapes.  \nBorn from collapsed stars to primordial fluctuations, they range 10⁻⁵–10¹⁰ M☉.  \nKerr metric governs rotating holes: horizon, ergosphere, no-hair.  \nWe detect them via X-ray disks, jets, GW mergers, and EHT shadows.  \nStill unknown: SMBH seeds, information loss, quantum interiors, dark-matter link.'

### 3.json output parser

<!-- it mainly use for give output in json format -->

In [29]:
# it is mainly use for give output in json foramat
print(type(parser))


<class 'langchain_core.output_parsers.string.StrOutputParser'>


In [24]:
from langchain_core.output_parsers import JsonOutputParser

In [25]:
parser2=JsonOutputParser()

In [33]:
t3=PromptTemplate(
    template="give the age,name and city of a fictional person \n {fromat_instruction}",
    input_variables=[],
    partial_variables={"fromat_instruction":parser2.get_format_instructions()}
)

In [35]:
p3=t3.format()

In [40]:
print(p3)

give the age,name and city of a fictional person 
 Return a JSON object.


In [38]:
print(llm.invoke(p3))

content='{"name":"Lila Moreau","age":34,"city":"New Lyon"}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 42, 'total_tokens': 60, 'completion_time': 0.069552107, 'completion_tokens_details': None, 'prompt_time': 0.018475718, 'prompt_tokens_details': None, 'queue_time': 0.280334569, 'total_time': 0.088027825}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c273f-9d37-7491-95ca-0276b12a3cee-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 42, 'output_tokens': 18, 'total_tokens': 60}


In [41]:
res=llm.invoke(p3)

In [42]:
final_result=parser2.parse(res.content)

In [43]:
print(final_result)

{'name': 'Elara Voss', 'age': 34, 'city': 'Lunaris'}
